In [1]:
import keras
import math
import pandas as pd
from tqdm import tqdm
import csv
import numpy as np

Using TensorFlow backend.


In [2]:
SUBM = 'V2.B'
MODEL = '/d3/caches/kaggle-integer-seq-v2/models/I.2/...'

In [3]:
# load stuff
dictionary = np.load('out/dictionary@tr=10.npy').item()
test_e_seq = np.load('out/test_e_seq@tr=10.npy').item()

In [4]:
# create reverse dictionary (ix -> word)
dic_ix_to_words = {}

for w, ix in dictionary.iteritems():
    dic_ix_to_words[ix] = w

In [5]:
X_test = test_e_seq.values()
test_ids = test_e_seq.keys()

In [6]:
batch_size = 320
orig_len = len(test_ids)
n_iter = int(math.ceil(float(orig_len) / batch_size))

In [7]:
model = keras.models.load_model(MODEL)

/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [8]:
predicted_words = []

for i in tqdm(range(n_iter)):
    
    X_test_batch = X_test[:batch_size]
    X_test = X_test[batch_size:]
    
    test_ids_batch = test_ids[:batch_size]
    test_ids = test_ids[batch_size:]

    # predict
    y_p = model.predict(np.array(X_test_batch, dtype=np.int32))
    y_p = np.argmax(y_p, axis=1)
    y_p = [dic_ix_to_words[x] for x in y_p]
    predicted_words.extend(y_p)

100%|██████████| 356/356 [05:18<00:00,  1.20it/s]


In [9]:
# save submission
res_df = pd.DataFrame({'"Id"': test_e_seq.keys(), '"Last"': predicted_words})
res_df.to_csv('submission-' +  SUBM + '.csv', index=False, quoting=csv.QUOTE_NONE)